# 本地调用tool工具-调用Agent执行本地命令

部分开发者需要给Client应用提供与本地命令交互的能力，比如在Client应用中调用Agent执行本地命令。并实现数据或文件的处理

##### 配置运行环境&导入Client应用

In [5]:
import os
import appbuilder

# AppBuilder Token
os.environ["APPBUILDER_TOKEN"] = "your api key"

# 应用为：智能问题解决者
app_id = "b9473e78-754b-463a-916b-f0a9097a8e5f"
app_client = appbuilder.AppBuilderClient(app_id)
conversation_id = app_client.create_conversation()

##### 创建一个能执行本地bash命令的tool工具

In [6]:
# 赋予应用一个执行本地命令的能力
def execute_local_command(cmd: str):
    import subprocess
    try:
        result = subprocess.check_output(cmd, shell=True).decode("utf-8")
        if result.strip() == "":
            return "命令执行成功，无返回值"
        return result
    except Exception as e:
        return str(e)


# 创建工具的描述：json_schema格式
tools = [
    {
        "type": "function",
        "function": {
            "name": "execute_local_command",
            "description": "可以在bash环境中，执行输入的指令，注意，一次只能执行一个原子命令。例如：ls",
            "parameters": {
                "type": "object",
                "properties": {
                    "cmd": {
                        "type": "string",
                        "description": "需要执行的指令",
                    },
                },
                "required": ["cmd"],
            },
        },
    }
]

- 我们再次询问应用，并给他赋予这个能力

In [ ]:
message_1 = app_client.run(
    conversation_id=conversation_id,
    query="请问当前文件夹下有哪些文件？如果没有test.txt文件，请新建一个test.txt文件，内容为：Hello World！",
    tools=tools
)
print("Agent的中间思考过程：")
print(message_1.content.events[-1].model_dump_json(indent=4))
print("Agent思考结束，等待我们上传本地结果\n")

**output**

- Agent的中间思考过程

```
Agent的中间思考过程：
{
    "code": 0,
    "message": "",
    "status": "interrupt",
    "event_type": "Interrupt",
    "content_type": "contexts",
    "detail": {
        "text": {
            "function_call": {
                "thought": "首先，我需要使用execute_local_command工具来执行ls命令，列出当前文件夹下的所有文件。然后，我需要检查输出中是否存在test.txt文件。如果不存在，我将再次使用execute_local_command工具来执行echo命令，创建一个新的test.txt文件，并写入内容'Hello World！'。",
                "name": "execute_local_command",
                "arguments": {
                    "cmd": "ls"
                },
                "usage": {
                    "prompt_tokens": 718,
                    "completion_tokens": 111,
                    "total_tokens": 829,
                    "name": "ERNIE-4.0-Turbo-8K",
                    "type": "plan"
                },
                "tool_call_id": "f60b241c-ddda-46aa-a2a6-b8bf1da2a424"
            },
            "used_tool": []
...
    ]
}
Agent思考结束，等待我们上传本地结果
```

- 大模型下发了调用本地函数的参数，我们使用这个参数调用本地函数

In [ ]:
tool_call = message_1.content.events[-1].tool_calls[-1]
tool_call_id = tool_call.id
tool_call_argument = tool_call.function.arguments
local_func_result = execute_local_command(**tool_call_argument)
print("No.1 local_func_result:\n {}\n".format(local_func_result))

**output**
```
No.1 local_func_result:
multi_tool_call.ipynb
multi_tool_call.py
multi_tool_call_with_handler.py
sdk_ knowledgebase.ipynb
sdk_trace.ipynb
simple_tool_call.ipynb
simple_tool_call.py
tmp.log
黑神话(悟空).pdf
```

- 向应用返回本地运行的结果，继续等待Agent的思考

In [ ]:
message_2 = app_client.run(
    conversation_id=conversation_id,
    tool_outputs=[{
        "tool_call_id": tool_call_id,
        "output": local_func_result
    }]
)

event_status = message_2.content.events[-1].status
if event_status != "interrupt":
    print("Agent 的回答是：\n")
    print(message_2.content.answer)
    quit()


print("Agent的中间思考过程：")
print(message_2.content.events[-1].model_dump_json(indent=4))

**output**
```
Agent的中间思考过程：
{
    "code": 0,
    "message": "",
    "status": "interrupt",
    "event_type": "Interrupt",
    "content_type": "contexts",
    "detail": {
        "text": {
            "function_call": {
                "thought": "根据execute_local_command工具的返回结果，当前文件夹下并没有'test.txt'文件。因此，我需要使用execute_local_command工具来执行'echo \"Hello World\" > test.txt'命令，以创建新的'test.txt'文件并写入内容'Hello World！'。",
                "name": "execute_local_command",
                "arguments": {
                    "cmd": "echo \"Hello World\" > test.txt"
                },
                "usage": {
                    "prompt_tokens": 1040,
                    "completion_tokens": 117,
                    "total_tokens": 1157,
                    "name": "ERNIE-4.0-Turbo-8K",
                    "type": "plan"
                },
                "tool_call_id": "9eaa3831-0d85-47eb-a39f-866ad2690c28"
            },
            "used_tool": [
...
            }
        }
    ]
}
```

- 大模型下发了调用本地函数的参数，我们使用这个参数调用本地函数

In [ ]:

tool_call = message_2.content.events[-1].tool_calls[-1]
tool_call_id = tool_call.id
tool_call_argument = tool_call.function.arguments
local_func_result = execute_local_command(**tool_call_argument)
print("No.2 local_func_result: {}\n".format(local_func_result))

**output**
```
No.2 local_func_result: 命令执行成功，无返回值
```

- 向应用返回本地运行的结果，完成应用的调用

In [ ]:
message_3 = app_client.run(
    conversation_id=conversation_id,
    tool_outputs=[{
        "tool_call_id": tool_call_id,
        "output": local_func_result
    }]
)

event_status = message_3.content.events[-1].status
if event_status != "interrupt":
    print("Agent 的回答是：\n")
    print(message_3.content.answer)
    quit()

print("Agent的中间思考过程：")
print(message_3.content.events[-1].model_dump_json(indent=4))

**output**
```
Agent的中间思考过程：
{
    "code": 0,
    "message": "",
    "status": "interrupt",
    "event_type": "Interrupt",
    "content_type": "contexts",
    "detail": {
        "text": {
            "function_call": {
                "thought": "用户的需求是列出当前文件夹下的所有文件，并在没有test.txt文件的情况下创建一个新的test.txt文件，内容为'Hello World！'。根据之前的执行结果，我们已经成功创建了test.txt文件并写入了内容。现在，我需要再次使用execute_local_command工具来执行ls命令，以确认test.txt文件已经成功创建，并且列出当前文件夹下的所有文件，以满足用户的完整需求。",
                "name": "execute_local_command",
                "arguments": {
                    "cmd": "ls"
                },
                "usage": {
                    "prompt_tokens": 1115,
                    "completion_tokens": 133,
                    "total_tokens": 1248,
                    "name": "ERNIE-4.0-Turbo-8K",
                    "type": "plan"
                },
                "tool_call_id": "eaf51384-b48c-41d2-965a-ca57efb4a6f1"
            },
            "used_tool": [
                {
                    "function_call": {
                        "thought": "首先，我需要使用execute_local_command工具来执行ls命令，列出当前文件夹下的所有文件。然后，我需要检查输出中是否存在test.txt文件。如果不存在，我将再次使用execute_local_command工具来执行echo命令，创建一个新的test.txt文件，并写入内容'Hello World！'。",
                        "name": "execute_local_command",
                        "arguments": {
                            "cmd": "ls"
                        }
                    },
                    "function_resp": "multi_tool_call.ipynb\nmulti_tool_call.py\nmulti_tool_call_with_handler.py\nsdk_ knowledgebase.ipynb\nsdk_trace.ipynb\nsimple_tool_call.ipynb\nsimple_tool_call.py\ntmp.log\n黑神话(悟空).pdf\n"
                },
                {
                    "function_call": {
                        "thought": "根据执行结果，当前文件夹下并没有test.txt文件，所以需要使用execute_local_command工具来执行echo命令，创建一个新的test.txt文件，并写入内容'Hello World！'。",
                        "name": "execute_local_command",
                        "arguments": {
                            "cmd": "echo 'Hello World！' > test.txt"
                        },
                        "usage": {
                            "prompt_tokens": 1025,
                            "completion_tokens": 96,
                            "total_tokens": 1121,
                            "name": "ERNIE-4.0-Turbo-8K",
                            "type": "plan"
                        },
                        "tool_call_id": "f43bac97-667d-40f0-a138-c6ade008bdef"
                    },
                    "function_resp": "命令执行成功，无返回值"
                }
            ]
        }
    },
    "usage": null,
    "tool_calls": [
        {
            "id": "eaf51384-b48c-41d2-965a-ca57efb4a6f1",
            "type": "function",
            "function": {
                "name": "execute_local_command",
                "arguments": {
                    "cmd": "ls"
                }
            }
        }
    ]
}
```

- 大模型下发了调用本地函数的参数，我们使用这个参数调用本地函数

In [ ]:
tool_call = message_3.content.events[-1].tool_calls[-1]
tool_call_id = tool_call.id
tool_call_argument = tool_call.function.arguments
local_func_result = execute_local_command(**tool_call_argument)
print("No.3 local_func_result: {}\n".format(local_func_result))

**output**
```
No.3 local_func_result: multi_tool_call.ipynb
multi_tool_call.py
multi_tool_call_with_handler.py
sdk_ knowledgebase.ipynb
sdk_trace.ipynb
simple_tool_call.ipynb
simple_tool_call.py
test.txt
tmp.log
黑神话(悟空).pdf
```

- 向应用返回本地运行的结果，完成应用的调用

In [ ]:
message_4 = app_client.run(
    conversation_id=conversation_id,
    tool_outputs=[{
        "tool_call_id": tool_call_id,
        "output": local_func_result
    }]
)
print("Agent 的回答是：\n")
print(message_4.content.answer)

**output**
```
Agent 的回答是：

当前文件夹下的文件包括：

- multi_tool_call.ipynb
- multi_tool_call.py
- multi_tool_call_with_handler.py
- sdk_knowledgebase.ipynb
- sdk_trace.ipynb
- simple_tool_call.ipynb
- simple_tool_call.py
- test.txt
- tmp.log
- 黑神话(悟空).pdf

由于当前文件夹下没有test.txt文件，因此已经成功创建了一个新的test.txt文件，并写入了内容'Hello World！'。现在，test.txt文件已经出现在文件夹中。
```

我们就完成了一个基于Client与自定义tool工具完成了一个 "请问当前文件夹下有哪些文件？如果没有test.txt文件，请新建一个test.txt文件，内容为：Hello World" 的操作